<a href="https://colab.research.google.com/github/sohaildua/PysparkLearning/blob/main/Aggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!pip install -U -q PyDrive --quiet 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 199 kB 50.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=d67bc0cb5f94f934956e718ca8ccf3aa52e427fafab07ed013319059e6d7d89c
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("aggrgate").getOrCreate()
spark

In [67]:
airbnb = spark.read.csv('nyc_air_bnb.csv', inferSchema=True, header=True)

In [45]:
airbnb.limit(4).toPandas()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,None,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194


In [68]:
airbnb.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: integer (nullable = true)



In [69]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

df = airbnb.withColumn("price", airbnb["price"].cast(IntegerType())) \
        .withColumn("minimum_nights", airbnb["minimum_nights"].cast(IntegerType())) \
        .withColumn("number_of_reviews", airbnb["number_of_reviews"].cast(IntegerType())) \
        .withColumn("reviews_per_month", airbnb["reviews_per_month"].cast(IntegerType())) \
        .withColumn("calculated_host_listings_count", airbnb["calculated_host_listings_count"].cast(IntegerType()))
#QA
print(df.printSchema())
df.limit(5).toPandas()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: integer (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)

None


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.0,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.0,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.0,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.0,1,0


In [21]:
airbnb.createOrReplaceTempView("airbnb")

from pyspark import sql

In [25]:
spark.sql("select neighbourhood_group,neighbourhood ,sum(Price) from airbnb GROUP BY neighbourhood_group,neighbourhood").show()

+-------------------+-----------------+----------+
|neighbourhood_group|    neighbourhood|sum(Price)|
+-------------------+-----------------+----------+
|             Queens|         Glendale|    4903.0|
|Morningside Heights|         40.81441|       6.0|
|               SoHo|         40.72057|       3.0|
|           Elmhurst|         40.74688|       7.0|
|           Brooklyn|  Windsor Terrace|   21822.0|
|       Williamsburg|         40.71383|       3.0|
|      Staten Island|   Fort Wadsworth|     800.0|
|         Douglaston|         40.75754|       1.0|
|             Queens|        Bayswater|    1487.0|
|        East Harlem|         40.79179|       3.0|
|             Queens|      Little Neck|     326.0|
|             Harlem|         40.82978|       1.0|
|             Queens|        Woodhaven|    5698.0|
|              Bronx|     Clason Point|    2368.0|
|              Bronx|      Castle Hill|     567.0|
|              Bronx|      Kingsbridge|    5455.0|
|      Staten Island|    Arden 

In [63]:
airbnb.groupBy("neighbourhood").agg(mean(col("price"))).toPandas()

,neighbourhood,avg(price)
0,Corona,59.171875
1,Richmondtown,78.000000
2,Prince's Bay,409.500000
3,Westerleigh,71.500000
4,Mill Basin,179.750000
...,...,...
378,40.76317,21.000000
379,40.86143,2.000000
380,40.61326,1.000000
381,40.77324,1.000000


In [72]:
airbnb.select("price", "minimum_nights").summary("count","min","25%","75%", "max").toPandas()

,summary,price,minimum_nights
0,count,48894,48894
1,min,-73.99986,0
2,25%,69.0,1.0
3,75%,175.0,5.0
4,max,Private room,Private room


In [ ]:
#pivot function to be studied

In [50]:
airbnb.count()

49079

In [57]:
airbnb.groupBy("host_name").agg(sum(col("number_of_reviews")).alias("x")).orderBy(col("x").desc()).show()

+---------+-------+
|host_name|      x|
+---------+-------+
|  Michael|11081.0|
|    David| 8100.0|
|     John| 7223.0|
|    Jason| 6522.0|
|     Alex| 6204.0|
|    Chris| 5028.0|
|     Anna| 4799.0|
|     Eric| 4733.0|
|   Daniel| 4723.0|
|    Sarah| 4579.0|
|    Kevin| 4549.0|
| Michelle| 4417.0|
|    Maria| 4354.0|
|   Andrew| 4150.0|
|  Jessica| 4074.0|
|     Mike| 3954.0|
|     Maya| 3938.0|
|  Anthony| 3877.0|
|      Amy| 3810.0|
|     Mark| 3780.0|
+---------+-------+
only showing top 20 rows



In [61]:
airbnb.summary("max","min").toPandas()

,summary,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,max,"獨一無二的紐約閣樓""","ﾏﾝﾊｯﾀﾝ､駅から徒歩4分でどこに行くのにも便利な場所!女性の方希望,ｷﾚｲなお部屋｡",呈刚,현선,Woodside,Woodside,West Village,Shared room,Shared room,Private room,Private room,99,9.66,Private room,99,365
1,min,"12 mins Manhattan""",1 Bed Apt in Utopic Williamsburg,"Heart of Greenwich Village""","very clean studio app""",194716858,2,-73.72247,-73.71299,-73.90783,-73.99986,0,0,-73.94134,0,0,0


In [66]:
airbnb.groupBy("host_name").agg(count(col("number_of_reviews")).alias("x")).orderBy(col("x").desc()).show(1)

+---------+---+
|host_name|  x|
+---------+---+
|  Michael|416|
+---------+---+
only showing top 1 row



In [80]:
airbnb.agg(mean(col("minimum_nights"))).toPandas()

,avg(minimum_nights)
0,7.128613


In [82]:
airbnb.filter("room_type IN ('Private room','Shared room')").groupBy("room_type").pivot("neighbourhood_group",['Manhattan', 'Brooklyn']).agg(avg(col('price'))).show()

+------------+------------------+-----------------+
|   room_type|         Manhattan|         Brooklyn|
+------------+------------------+-----------------+
| Shared room| 89.06903765690376|50.52784503631961|
|Private room|116.05400302114803|76.47234042553191|
+------------+------------------+-----------------+

